# 車の検出とバウンディングボックスの配置

このノートブックでは、Yoloモデルによって検出された車の周りにボックスを描く方法を学びます。

まず、YOLOパッケージをインポートします。

In [ ]:
# このLab用に設計されたWorkbenchイメージを使用していない場合は、以下の行をアンコメントして必要なパッケージをインストールしてください。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

from ultralytics import YOLO
from PIL import Image

In [ ]:
# オブジェクト検出用にYOLOv8mモデルを使用します

model = YOLO("yolov8m.pt")

In [ ]:
# テスト画像に対するモデル予測結果を取得します

img = "images/carImage0.jpg"
results = model.predict(img)

In [ ]:
# YOLOでは1枚の画像だけでなく、画像の配列を提出することができ、結果の配列を得ることができます。
# 今回は1枚の画像だけを提出したため、結果の配列から最初の（唯一の）要素を取得する必要があります。

result = results[0]

In [ ]:
# 検出されたボックスの数を確認します

len(result.boxes)

In [ ]:
# ボックスの詳細を解析します

box = result.boxes[0]
print("オブジェクトの種類:", box.cls)
print("座標:", box.xyxy)
print("確率:", box.conf)

In [ ]:
# Tensorから実際の値を取り出します

cords = box.xyxy[0].tolist()
class_id = box.cls[0].item()
conf = box.conf[0].item()
print("オブジェクトの種類:", class_id)
print("座標:", cords)
print("確率:", conf)

In [ ]:
# COCOはYOLOモデルがトレーニングされたデータセットです。検出するオブジェクトはクラスで整理されています。これが「オブジェクトの種類」フィールドで得られた情報です。
# YOLOv8の結果オブジェクトには、これらのクラス名の'名前'プロパティも含まれています。

print(result.names)


クラス番号「2」は「車」オブジェクトに対応しています。つまり、結果のバウンディングボックスは検出された「車」に対応しています。
それでは、画像にボックスを描きましょう！


In [ ]:
# まず、座標をリストに入れ、四捨五入します。
# 次に、result.names辞書を使用して検出されたオブジェクトクラスの名前をIDで取得します。

cords = box.xyxy[0].tolist()
cords = [round(x) for x in cords]
class_id = result.names[box.cls[0].item()]
conf = round(box.conf[0].item(), 2)
print("オブジェクトの種類:", class_id)
print("座標:", cords)
print("確率:", conf)

In [ ]:
# ボックス全体をループして情報を抽出しましょう。

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("オブジェクトの種類:", class_id)
  print("座標:", cords)
  print("確率:", conf)
  print("---")

In [ ]:
# 画像にボックスを描き、クラス名と確率（モデルが検出にどれほど自信があるか）を表示します。

Image.fromarray(result.plot()[:,:,::-1])

それでは、前のノートブックでテストした複数の車が写っている画像（carImage4.jpg）に戻り、YOLOが画像内のすべての「車」を識別できるか確認しましょう。

In [ ]:
# これは前のセルで使用したのと同じコードですが、1回で実行します

results = model.predict("images/carImage4.jpg")

result = results[0]

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("オブジェクトの種類:", class_id)
  print("座標:", cords)
  print("確率:", conf)
  print("---")

Image.fromarray(result.plot()[:,:,::-1])

YOLOモデルは画像の「遠くにある」車を見逃してしまったことがわかります。しかし全体として、モデルはこの画像内の複数の車を特定するのに素晴らしい仕事をしました。そして、重要なことに、特定された車は「バウンディングボックス」に囲まれています！

YOLOモデルによって認識された車の周りにバウンディングボックスを配置できるようになったので、車の「衝突」を特定するためにYOLOモデルを再トレーニングできます。

**ノートブック `04-03-model-retraining.ipynb` を開いてください。**